In [1]:
from booknlp.booknlp import BookNLP

model_params={
		"pipeline":"entity,quote,supersense,event,coref", 
		"model":"big"
	}
	
booknlp=BookNLP("en", model_params)




using device cpu
{'pipeline': 'entity,quote,supersense,event,coref', 'model': 'big'}
--- startup: 27.868 seconds ---


In [2]:
# Input file to process
input_file="a_little_princess.txt"

# Output directory to store resulting files in
output_directory="./output_dir/princess/"

# File within this directory will be named ${book_id}.entities, ${book_id}.tokens, etc.
book_id="princess"

booknlp.process(input_file, output_directory, book_id)


--- spacy: 7.132 seconds ---
--- entities: 42.034 seconds ---
--- quotes: 0.042 seconds ---
--- attribution: 86.555 seconds ---
--- name coref: 0.047 seconds ---
--- coref: 49.948 seconds ---
--- TOTAL (excl. startup): 185.894 seconds ---, 81696 words


In [4]:
import json
from collections import Counter

def proc(filename):
    with open(filename) as file:
        data=json.load(file)
    return data

data=proc("./output_dir/princess/princess.book")

In [ ]:
def get_counter_from_dependency_list(dep_list):
    counter=Counter()
    for token in dep_list:
        term=token["w"]
        tokenGlobalIndex=token["i"]
        counter[term]+=1
    return counter

for character in data["characters"]:

    agentList=character["agent"]
    patientList=character["patient"]
    possList=character["poss"]
    modList=character["mod"]

    character_id=character["id"]
    count=character["count"]

    referential_gender_distribution=referential_gender_prediction="unknown"

    if character["g"] is not None and character["g"] != "unknown":
        referential_gender_distribution=character["g"]["inference"]
        referential_gender=character["g"]["argmax"]

    mentions=character["mentions"]
    proper_mentions=mentions["proper"]
    max_proper_mention=""

    # just print out information about named characters
    if len(mentions["proper"]) > 0:
        max_proper_mention=mentions["proper"][0]["n"]

        print(character_id, count, max_proper_mention, referential_gender)

        print()
        printTop=100
        for k, v in get_counter_from_dependency_list(possList).most_common(printTop):
            print("\tposs\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(agentList).most_common(printTop):
            print("\tagent\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(patientList).most_common(printTop):
            print("\tpatient\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(modList).most_common(printTop):
            print("\tmod\t%s %s" % (v,k))
        print()

In [6]:
import pandas as pd

df_tokens = pd.read_csv("./output_dir/princess/princess.tokens", delimiter="\t")
df_entities = pd.read_csv("./output_dir/princess/princess.entities", delimiter="\t")

#create dictionary with the shape [sentence_ID][token_ID_within_sentence][word]
sentence_and_token_ID_to_word_dict = {}
for character in data["characters"]:
    character_id=character["id"]
    mentions=character["mentions"]
    proper_mentions=mentions["proper"]
    if len(mentions["proper"]) > 0:
        max_proper_mention=mentions["proper"][0]["n"]
    else:
        max_proper_mention = character_id

    unique_start_token_ids = df_entities.loc[df_entities['COREF'] == character_id, 'start_token']
    #print(unique_start_token_ids)
    sentence_ids = df_tokens[df_tokens['token_ID_within_document'].isin(unique_start_token_ids)][['token_ID_within_document', 'token_ID_within_sentence', 'sentence_ID']]
    sentence_ids = sentence_ids.reset_index(drop=True)

    for index, row in sentence_ids.iterrows():
        sentence_ID = row['sentence_ID']
        token_ID = row['token_ID_within_sentence']
        combined_sentence_and_token_ID = str(sentence_ID) +":"+ str(token_ID)
        sentence_and_token_ID_to_word_dict[combined_sentence_and_token_ID] = max_proper_mention
    

In [7]:
character = data["characters"][1]
character_id=character["id"]

#get all sentencs where COREF = character_id
unique_start_token_ids = df_entities.loc[df_entities['COREF'] == character_id, 'start_token']
print(unique_start_token_ids)
sentence_ids = df_tokens[df_tokens['token_ID_within_document'].isin(unique_start_token_ids)][['token_ID_within_document', 'token_ID_within_sentence', 'sentence_ID']]
sentence_ids = sentence_ids.reset_index(drop=True)
print(sentence_ids)

235       1249
247       1308
285       1594
287       1600
288       1604
         ...  
12707    80512
12708    80519
12709    80525
12717    80585
12719    80605
Name: start_token, Length: 1084, dtype: int64
      token_ID_within_document  token_ID_within_sentence  sentence_ID
0                         1249                        38           47
1                         1308                        11           50
2                         1594                         5           65
3                         1600                         0           66
4                         1604                         4           66
...                        ...                       ...          ...
1079                     80512                         4         4425
1080                     80519                        11         4425
1081                     80525                        17         4425
1082                     80585                        10         4427
1083               

In [8]:
from allennlp.predictors.predictor import Predictor
SRL_MODEL_PATH = "srl.tar.gz"
predictor_srl = Predictor.from_path(SRL_MODEL_PATH)

In [9]:
import utils
import spacy
nlp = spacy.load("en_core_web_sm")

In [10]:
def check_for_arg(n, arg_name, dict_name, sentence_ID):
    if arg_name in n["tags"]:
        index = n['tags'].index(arg_name)
        if str(sentence_ID)+":"+str(index) in dict_name.keys():
            return dict_name[str(sentence_ID)+":"+str(index)]
        else:
            return None
    else:
        return None
    


In [11]:
columns = ['sentence_ID', 'verb', 'agent', 'patient']
srl_results_df = pd.DataFrame(columns=columns)

# for each mention in our coreference cluster do the following
for index, row in sentence_ids.iterrows():
    token_ID_within_document = row['token_ID_within_document']
    #word = df_tokens.loc[df_tokens['token_ID_within_document'] == token_ID_within_document, 'word'].values[0]

    sentence_ID = df_tokens.loc[df_tokens['token_ID_within_document'] == token_ID_within_document, 'sentence_ID'].values[0]
    words_list = df_tokens.loc[df_tokens['sentence_ID'] == sentence_ID, 'word'].tolist()
    token_ID_within_sentence = row['token_ID_within_sentence']
    sentence_for_srl = ' '.join(words_list)
    predictions_srl = predictor_srl.predict(sentence_for_srl)

    for n in predictions_srl['verbs']:
        if token_ID_within_sentence < len(n['tags']):
            tag = n['tags'][token_ID_within_sentence]
            if "B-ARG0" in tag:

                word = check_for_arg(n, "B-ARG1", sentence_and_token_ID_to_word_dict, sentence_ID)
                if word != None:
                    new_row = pd.DataFrame({'sentence_ID': [sentence_ID], 'verb': [utils.lemmatize(n['verb'], nlp)], 'agent': ["protag"], 'patient': [word]})
                    srl_results_df = pd.concat([srl_results_df, new_row], ignore_index=True)
            elif "B-ARG1" in tag:
                word = check_for_arg(n, "B-ARG0", sentence_and_token_ID_to_word_dict, sentence_ID)
                if word != None:
                    new_row = pd.DataFrame({'sentence_ID': [sentence_ID], 'verb': [utils.lemmatize(n['verb'], nlp)], 'agent': [word], 'patient': ["protag"]})
                    srl_results_df = pd.concat([srl_results_df, new_row], ignore_index=True)

srl_results_df.to_csv('srl_output_princess.csv', index=False)
del srl_results_df